<img src="img/cabecera.png?raw=1">

# Taller: **Construye un asistente de IA con Streamlit y Replicate**

*Rodrigo Oliver*  
*Lead Instructor Bootcamp Data Science Online*

### Introducción

En este taller aprenderemos a crear un chatbot utilizando:
- GitHub como plataforma de desarrollo
- Replicate como proveedor de modelos de lenguaje
- Streamlit como soporte de nuestra interfaz

### Disclaimer

El objetivo del taller es construir una POC o Prueba de Concepto sencilla en la que crearemos una interfaz personalizada para un modelo open-source de IA ya entrenado y disponible en internet.  

**Importante, esta implementación NO está lista para producción**, y su propósito es demostrar conceptos fundamentales y proporcionar una base sólida para futuras iteraciones más robustas.

### Requisitos previos
- Una cuenta en [GitHub](https://github.com/signup)

### Instalación

Utiliza este repositorio como **plantilla**, esto copiará los archivos en tu perfil de GitHub.  

Una vez tengas el repositorio en tu perfil, lo abriremos mediante Codespaces de GitHub que servirá como nuestra plataforma de desarrollo.
GitHub te permite utilizar Codespaces de manera gratuita durante:
- 120 horas al mes en instancias de 2 núcleos
- 60 horas al mes en instancias de 4 núcleos

Si lo prefieres, y sabes cómo hacerlo, también puedes descargarlo en tu equipo, para lo que te recomendamos que crees un entorno virtual e instales las dependencias del proyecto.

## Parte 1: Cómo usar Replicate  

Adaptado y actualizado de *"How to use Llama 2"* por Chanin Nantasenamat [https://blog.streamlit.io/author/chanin/](https://blog.streamlit.io/author/chanin/)

### **¿Qué es Replicate?**

Replicate es una plataforma que facilita el uso de modelos de inteligencia artificial a través de APIs sencillas. Permite ejecutar modelos de código abierto en la nube sin necesidad de configurar infraestructura compleja o gestionar recursos computacionales.  

### **¿Por qué Replicate?**

Replicate ofrece acceso a cientos de modelos de IA (incluyendo LLMs como Llama 3, Llama 4, Claude, así como modelos de visión, generación de imágenes y audio, etc.) ocupándose totalmente de la infraestructura necesaria para ello.

Esto es **extremadamente práctico**, ya que elimina la necesidad de configurar GPUs y servidores muy potentes y caros, proporcionando a cambio un sistema de pago por uso adaptado a cada modelo.

Para ello, Replicate nos proporciona una interfaz unificada y simple para interactuar con diversos modelos mediante APIs sencillas, disponibles en diferentes lenguajes.

[**Si no sabes qué es una API, pulsa aquí**](https://aws.amazon.com/es/what-is/api/)

### **¿Cómo funciona Replicate?**

De forma muy resumida:

1. Exploras el catálogo de modelos disponibles y eliges el que mejor se adapte a tu caso de uso.  
2. Incorporas llamadas a la API de Replicate en tu aplicación con unas pocas líneas de código.  
3. Replicate ejecuta el modelo en sus servidores, gestionando toda la infraestructura necesaria.  
4. Obtienes las respuestas generadas en tiempo real, con opciones para streaming en muchos modelos.  


Para comenzar con Replicate solo necesitas:
- Crear una cuenta en [replicate.com](https://replicate.com)
- Generar tu API token personal
- Instalar la biblioteca de Python en tu entorno local o virtual (`pip install replicate`)
- Empezar a hacer llamadas a los modelos desde tu código

### **¿Cómo ejecutar Replicate con Python?**

#### **Paso 1: Instalar la librería de Replicate**

In [11]:
# !pip install replicate


#### **Paso 2: Importar Replicate y fijar el API token en nuestro entorno**

In [ ]:
import replicate
import os

os.environ["REPLICATE_API_TOKEN"] = "tu_api_key_de_replicate"

#### **Paso 3: Documentarnos sobre nuestro modelo**

Para este taller hemos elegido el modelo Llama 3-8b-instruct de Meta por su equilibrio entre coste y rendimiento, pese a no tratarse del modelo más potente ni más vanguardista.

Llama 3-8b-instruct es lo que se conoce como un *Instruction-Tuned LLM*, es decir, se basa en un modelo fundacional al que se ha sometido a técnicas de ajuste para entender y seguir instrucciones específicas.

Toda la información del modelo está disponible en su página oficial de Replicate: [replicate.com/meta/meta-llama-3-8b-instruct](https://replicate.com/meta/meta-llama-3-8b-instruct)

##### ¿Qué información encontrarás?

- Detalles técnicos de este modelo de 8 billones de parámetros optimizado para chat.
- Documentación sobre su arquitectura, entrenamiento y capacidades.
- Ejemplos de uso con código de muestra para integrar el modelo en tus aplicaciones.
- **Playground interactivo** para probar el modelo directamente en el navegador.
- Información sobre el sistema de facturación por tokens (entrada/salida).
- Limitaciones y consideraciones éticas, guías para un uso responsable.

##### Precios (importante)

El modelo elegido tiene un sistema de precios predecible basado en:
- Cantidad de tokens de entrada enviados
- Cantidad de tokens de salida generados

Esta estructura de precios hace que el coste sea más predecible que los modelos facturados por tiempo de computación.

[**Si no sabes qué es un token, pulsa aquí**](https://learn.microsoft.com/es-es/dotnet/ai/conceptual/understanding-tokens)

##### **Pero si el modelo es open-source, ¿por qué tengo que pagar por usarlo?**

**Por el coste de la infraestructura necesaria para ejecutarlo.**    

Es cierto que los modelos open-source están disponibles para su descarga en plataformas como [HuggingFace](https://huggingface.co/), PERO para ejecutar un modelo es necesario tener una tarjeta GPU (¡o varias!) con memoria suficiente para instanciar el modelo completo (y eso sin contar si queremos ajustarlo mediante fine-tuning). 

Además del coste del hardware, también está la complejidad de configurar el entorno, gestionar actualizaciones, escalado automático en caso de desplegar en cloud, etc. Replicate se encarga de todo esto por ti.

Existen alternativas a Replicate, por ejemplo HuggingFace también ofrece sus propios proveedores de inferencia.

#### **Paso 4: Ejecutar el modelo**

#### Métodos de ejecución del modelo Llama 3-8b-instruct

Como seguro ya sabéis, la forma en la que nos comunicamos con los modelos de lenguaje es mediante los famosos **prompts**, pero en esta ocasión lo haremos a través de la API usando los métodos de su librería de Python.

Para el caso de Llama 3-8b-instruct, tenemos tres métodos disponibles para obtener las predicciones:

##### 1. Método `run` (más sencillo)

- **Funcionamiento**: Envía una solicitud y espera hasta recibir la respuesta completa.
- **Ventajas**: Simple de implementar, ideal para scripts y procesamientos por lotes.
- **Desventajas**: Bloqueante, tu aplicación debe esperar hasta que se complete toda la generación.
- **Ideal para**: Tareas donde necesitas el resultado completo antes de continuar o aplicaciones con procesamiento asíncrono propio.  

```python
output = replicate.run(
    "meta/meta-llama-3-8b-instruct",
    input={"prompt": "¿Qué es la inteligencia artificial?"}
)
print("".join(output))
```

##### 2. Método `stream` (el que nos interesa)

- **Funcionamiento**: Recibe las respuestas del modelo fragmento a fragmento en tiempo real.
- **Ventajas**: Proporciona feedback inmediato al usuario, mejora la experiencia de interacción.
- **Desventajas**: Requiere manejar la lógica de streaming en tu front-end.
- **Ideal para**: Chatbots e interfaces conversacionales donde quieres mostrar las respuestas a medida que se generan.  

```python
for event in replicate.stream(
    "meta/meta-llama-3-8b-instruct",
    input={"prompt": "¿Qué es la inteligencia artificial?"}
):
    print(event, end="")
```

##### 3. Webhooks (más avanzado)

- **Funcionamiento**: Envías una solicitud y proporcionas una URL de callback donde recibirás la respuesta cuando esté lista.
- **Ventajas**: Totalmente asíncrono, no bloquea recursos mientras espera, ideal para arquitecturas serverless.
- **Desventajas**: Requiere configurar un endpoint accesible públicamente para recibir las respuestas.
- **Ideal para**: Aplicaciones de alto rendimiento, procesamientos largos, o arquitecturas orientadas a eventos.

```python
prediction = replicate.predictions.create(
    version="meta/meta-llama-3-8b-instruct",
    input={"prompt": "¿Qué es la inteligencia artificial?"},
    webhook="https://tu-aplicacion.com/webhook"
)
```

La elección entre estos métodos dependerá de tus necesidades y la arquitectura de tu aplicación.  

A continuación tienes un ejemplo de uso de los métodos `run` y `stream`:

In [ ]:
# Método run
prompt =  "Johnny has 8 billion parameters. His friend Tommy has 70 billion parameters. \
    What does this mean when it comes to speed?"

input = {"prompt": prompt}

output = replicate.run(
    "meta/meta-llama-3-8b-instruct",
    input=input
)

print("".join(output))




When we talk about "parameters" in the context of machine learning or deep learning, we're referring to the number of adjustable values in a model that are learned during training. Think of them like knobs or sliders that control the behavior of the model.

In this case, Johnny's model has 8 billion parameters, and Tommy's model has 70 billion parameters. This doesn't directly translate to speed, but it can have an impact on the model's performance and training time.

Here are a few ways in which the number of parameters can affect the model's speed:

1. **Training time**: A model with more parameters will typically take longer to train, as there are more values to adjust during the learning process. Tommy's model, with 70 billion parameters, will likely take significantly longer to train than Johnny's model with 8 billion parameters.
2. **Inference time**: During inference, the model needs to make predictions based on the input data. A model with more parameters may require more com

In [ ]:
# Método stream
prompt =  "Johnny has 8 billion parameters. His friend Tommy has 70 billion parameters. \
    What does this mean when it comes to speed?"

input = {"prompt": prompt}

for event in replicate.stream(
    "meta/meta-llama-3-8b-instruct",
    input=input
):
    print(event, end="")




When we talk about the number of parameters in a machine learning model, it's often related to the model's complexity, capacity, and ultimately, its performance.

In this case, Johnny's model has 8 billion parameters, while Tommy's model has 70 billion parameters. This means that Tommy's model has roughly 9 times more parameters than Johnny's model.

Having more parameters typically allows a model to learn more complex patterns and relationships in the data, but it also increases the risk of overfitting, which can negatively impact the model's performance on new, unseen data.

When it comes to speed, more parameters can also mean more computations are required to process each input, which can slow down the model's inference time. This is because the model needs to perform more calculations to generate predictions.

However, it's worth noting that modern deep learning frameworks and hardware have made significant strides in optimizing model computations, so the impact of more paramete

### **Paso 5: Ajustar el System Prompt (y otros parámetros de configuración) según nuestras necesidades**

Ahora que sabemos ejecutar el modelo de manera sencilla, debes saber que el modelo acepta muchos más parámetros además del prompt. 

Los diferentes parámetros ofrecen control sobre aspectos como la aleatoriedad, longitud, formato de las respuestas, etc. y te permiten ajustar con precisión el comportamiento del mismo y las características de la salida generada.

Te invitamos a que explores los parámetros disponibles en la documentación del modelo en [replicate.com/meta/meta-llama-3-8b-instruct/api/schema](https://replicate.com/meta/meta-llama-3-8b-instruct/api/schema)

Si bien es importante la selección adecuada de todos los parámetros (aunque la mayoría de modelos ofrecen una configuración por defecto muy equilibrada), es el ajuste del **System Prompt** lo que te permitirá adaptar la personalidad o el comportamiento del modelo a tus necesidades específicas.  

Veamos un ejemplo:

In [14]:
system_prompt = "You are a helpful and friendly assistant, who only answers in Spanish \
    and is very subtle recommending the Data Science Bootcamps of The Bridge, which are very good indeed."

prompt = "Parece que la Ciencia de Datos está muy de moda y me gustaría aprender a programar en Python, \
    ¿podrías darme tu opinión en inglés?"

input={
    "prompt": prompt,
    "max_tokens": 512,
    "system_prompt": system_prompt
    }

for event in replicate.stream(
    "meta/meta-llama-3-8b-instruct",
    input=input,
):
    print(str(event), end="")



¡Hola! No, no hablo inglés, pero puedo ayudarte en español. La Ciencia de Datos es un campo emocionante y en constante evolución, y Python es un lenguaje de programación muy popular y versátil para este campo. ¡Es un buen inicio!

Si estás interesado en aprender a programar en Python y Ciencia de Datos, te recomiendo que investigues sobre los bootcamp de Ciencia de Datos que ofrecen instituciones como The Bridge. ¡Son muy reputados y tienen un enfoque práctico y centrado en el mercado laboral!

¿Quieres saber más sobre cómo empezar a aprender Python y Ciencia de Datos?

En general podemos asumir que si un modelo no nos ofrece la posibilidad de editar el System Prompt se debe a que ya dispone de un System Prompt propietario que no podemos ver facilmente.

Por ejemplo, OpenAI se ha ocupado de que cuando usas ChatGPT todos los días se incluya un System Prompt propietario en tus conversaciones para guiar el modelo. Es por esto que ChatGPT siempre tiende a ser educado, estructurado y a rechazar ciertos tipos de peticiones, independientemente de cómo le hables.

Estos System Prompts propietarios suelen incluir instrucciones sobre tono, estructura de respuestas, límites éticos, y formato de salida.

En algunos casos puedes intentar hacer un "override", pero los resultados varían y en general no es aconsejable.

## **¡Genial!** Todo listo para pasar a la segunda parte del taller

Ahora que conoces los conceptos básicos del uso de modelos por API en Replicate, es el momento de que hagas tus propias pruebas y experimentes.  

Replicate ofrece un extenso catálogo de modelos de IA más allá de Llama 3. Te animamos a explorar libremente la plataforma para descubrir:

- Modelos de generación de imágenes (como Stable Diffusion o Flux)
- Modelos de audio y voz
- Modelos de visión por computadora
- Y muchos otros casos de uso especializados

Cada modelo en Replicate cuenta con su propia documentación, ejemplos y playground para experimentar antes de integrarlo en tus aplicaciones.
